In [2]:
import cv2
import numpy as np
import dlib
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array

# Load the trained CNN model
model = tf.keras.models.load_model('eyes_open_closed_model.h5')

# Initialize dlib's face detector and landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')  # Path to the pre-trained model

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Variables for smoothing predictions
CONSEC_FRAMES = 10  # The number of consecutive frames for drowsiness detection
drowsiness_counter = 0
frame_window = 5  # Number of frames to smooth the predictions over
eye_state_history = []  # Store recent predictions (Open/Closed)

# Function to preprocess the frame for the model
def preprocess_eye_region(eye_region):
    # Resize the eye region to 90x90
    image = cv2.resize(eye_region, (90, 90))
    
    # Convert to grayscale
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Normalize the image
    image = img_to_array(image) / 255.0
    
    # Expand dimensions to fit the model input shape (batch_size, height, width, channels)
    image = np.expand_dims(image, axis=-1)  # Add channel dimension
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    
    return image

# Function to predict if the eyes are open or closed
def predict_eye_state(eye_region):
    # Preprocess the eye region
    preprocessed_image = preprocess_eye_region(eye_region)
    
    # Make a prediction
    prediction = model.predict(preprocessed_image)
    
    # Return prediction: 'Closed' if prediction[0][0] > prediction[0][1], else 'Open'
    if prediction[0][0] > prediction[0][1]:
        return 'Closed'
    else:
        return 'Open'

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces using dlib's face detector
    faces = detector(gray)

    for face in faces:
        # Get the facial landmarks
        landmarks = predictor(gray, face)

        # Draw a rectangle around the face
        x, y, w, h = (face.left(), face.top(), face.width(), face.height())
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Blue box for the face

        # Get the coordinates of the left and right eyes
        left_eye_points = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(36, 42)]
        right_eye_points = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(42, 48)]

        # Get the bounding box for the left eye
        left_eye_box = cv2.boundingRect(np.array(left_eye_points))
        right_eye_box = cv2.boundingRect(np.array(right_eye_points))

        # Extract the eye regions from the frame
        left_eye_region = frame[left_eye_box[1]:left_eye_box[1] + left_eye_box[3], left_eye_box[0]:left_eye_box[0] + left_eye_box[2]]
        right_eye_region = frame[right_eye_box[1]:right_eye_box[1] + right_eye_box[3], right_eye_box[0]:right_eye_box[0] + right_eye_box[2]]

        # Predict the eye state (open/closed) for both eyes
        left_eye_state = predict_eye_state(left_eye_region)
        right_eye_state = predict_eye_state(right_eye_region)

        # Append the latest prediction to the history
        eye_state_history.append((left_eye_state, right_eye_state))

        # Keep only the latest `frame_window` predictions in the history
        if len(eye_state_history) > frame_window:
            eye_state_history.pop(0)

        # Determine the number of frames where both eyes are closed
        closed_both_eyes = sum(1 for state in eye_state_history if state[0] == 'Closed' and state[1] == 'Closed')

        # If both eyes are closed for a certain percentage of frames, consider it drowsiness
        if closed_both_eyes >= (frame_window // 2):  # 50% of the frames
            drowsiness_counter += 1
            if drowsiness_counter >= CONSEC_FRAMES:
                cv2.putText(frame, "Drowsy", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        else:
            drowsiness_counter = 0  # Reset the counter if eyes are open

        # Draw bounding boxes around the eyes and display the predictions
        cv2.rectangle(frame, (left_eye_box[0], left_eye_box[1]), 
                      (left_eye_box[0] + left_eye_box[2], left_eye_box[1] + left_eye_box[3]), 
                      (0, 255, 0), 2)  # Green box for left eye
        cv2.putText(frame, f'{left_eye_state}', (left_eye_box[0], left_eye_box[1] - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        cv2.rectangle(frame, (right_eye_box[0], right_eye_box[1]), 
                      (right_eye_box[0] + right_eye_box[2], right_eye_box[1] + right_eye_box[3]), 
                      (0, 255, 0), 2)  # Green box for right eye
        cv2.putText(frame, f'{right_eye_state}', (right_eye_box[0], right_eye_box[1] - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Eye State Detection (Dlib + CNN)', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━